In [35]:
%matplotlib inline

import datacube
import numpy as np
from matplotlib import pyplot as plt
from deafrica_tools.plotting import display_map

In [36]:
dc = datacube.Datacube(app="rainfall_chirps")

In [38]:
lat, lon =  16.625, -14.875 

buffer_lat, buffer_lon = 1, 1

time_period = ('2013-03-01', '2020-12-31')

output_crs = 'epsg:6933'

#join lat,lon,buffer to get bounding box
lon_range = (lon - buffer_lon, lon + buffer_lon)
lat_range = (lat + buffer_lat, lat - buffer_lat)

In [39]:
display_map(lon_range, lat_range)

In [40]:
dc.list_products().loc[dc.list_products()['name'].str.contains('chirps')]

,name,description,license,default_crs,default_resolution
name,,,,,
rainfall_chirps_daily,rainfall_chirps_daily,Rainfall Estimates from Rain Gauge and Satelli...,None,EPSG:4326,"(-0.05000000074505806, 0.05000000074505806)"
rainfall_chirps_monthly,rainfall_chirps_monthly,Rainfall Estimates from Rain Gauge and Satelli...,None,EPSG:4326,"(-0.05000000074505806, 0.05000000074505806)"


In [41]:
measurements = dc.list_measurements()
measurements.loc["rainfall_chirps_monthly"]

,name,dtype,units,nodata,aliases,flags_definition
measurement,,,,,,
rainfall,rainfall,float32,mm,-9999.0,NaN,NaN


In [43]:
ds_rf_month = dc.load(product='rainfall_chirps_monthly',
                time=('2013-03-01', '2020-12-31'),
                y = lat_range,
                x = lon_range,
                resolution=(-2000, 2000),
                output_crs=output_crs)

print(ds_rf_month)

<xarray.Dataset> Size: 5MB
Dimensions:      (time: 94, y: 124, x: 97)
Coordinates:
  * time         (time) datetime64[ns] 752B 2013-03-16T11:59:59.500000 ... 20...
  * y            (y) float64 992B 2.215e+06 2.213e+06 ... 1.971e+06 1.969e+06
  * x            (x) float64 776B -1.531e+06 -1.529e+06 ... -1.339e+06
    spatial_ref  int32 4B 6933
Data variables:
    rainfall     (time, y, x) float32 5MB 0.1225 0.1229 0.1229 ... 0.3683 0.3683
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref


In [45]:
# Calculate the mean rainfall over the area for each time step
rainfall_timeseries = ds_rf_month['rainfall'].mean(dim=["x", "y"]).to_dataframe()


In [47]:
# Reset index to have a flat DataFrame format and save to CSV
rainfall_timeseries.reset_index().to_csv("rainfall_monthly_2013-2020.csv", index=False)
